In [1]:
import csv
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
def stat_unique(data: pd.DataFrame, key):
    if key is None:
        print('Total length: {}'.format(len(data)))
    elif isinstance(key, str):
        print('Number of unique {}: {}'.format(key, len(data[key].unique())))
        return len(data[key].unique())
    elif isinstance(key, list):
        print('Number of unique [{}]: {}'.format(','.join(key), len(data.drop_duplicates(key, keep='first'))))
        return len(data.drop_duplicates(key, keep='first'))

In [3]:
raw_data = pd.read_csv('../../data/assistment2009/non_skill_builder_data_new.csv', encoding = 'utf-8', dtype={'skill_id': str})
raw_data.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,20223588,245748,77759,12914,12914,1,1.0,1,57310.0,tutor,...,0,0,57310,12914,20935.0,NaN,0.0,NaN,1,1
1,20223595,245748,77759,15320,15320,1,1.0,1,88691.0,tutor,...,0,0,88691,15320,22385.0,NaN,0.0,NaN,2,2
2,20223601,245748,77759,14529,14529,1,1.0,1,43748.0,tutor,...,0,0,43748,14529,20735.0,NaN,0.0,NaN,3,3
3,20223926,245698,77912,1159,1159,1,0.0,2,45587.0,tutor,...,0,0,53599,1159,NaN,3,0.0,NaN,1,1
4,20223934,245698,77912,1647,1647,1,1.0,1,90176.0,tutor,...,0,0,90176,1647,3438.0,NaN,0.0,NaN,1,1


In [4]:
raw_data = raw_data.rename(columns={'user_id': 'student_id',
                                    'problem_id': 'question_id',
                                    'skill_id': 'knowledge_id',
                                    'skill_name': 'knowledge_name',
                                    })
all_data = raw_data.loc[:, ['student_id', 'question_id', 'knowledge_id', 'knowledge_name', 'correct']].dropna()

In [5]:
stat_unique(all_data, None)
a=stat_unique(all_data, ['student_id', 'question_id'])
b=stat_unique(all_data, 'student_id')
c=stat_unique(all_data, 'question_id')
d=stat_unique(all_data, 'knowledge_id')

Total length: 562268
Number of unique [student_id,question_id]: 526434
Number of unique student_id: 8043
Number of unique question_id: 6630
Number of unique knowledge_id: 525


In [33]:
def format_knowledge_id(x):
    # Chuyển giá trị về chuỗi để xử lý nhất quán
    s = str(x)
    # Nếu chứa dấu phẩy, lấy phần tử đầu tiên
    if ',' in s:
        return int(s.split(',')[0])
    else:
        return int(s)

# Áp dụng hàm cho cột knowledge_id
all_data['knowledge_id'] = all_data['knowledge_id'].apply(format_knowledge_id)

## Filter data

In [34]:
selected_data = all_data

In [35]:
# filter questions
n_students = selected_data.groupby('question_id')['student_id'].count()
question_filter = n_students[n_students < 50].index.tolist()
print(f'filter {len(question_filter)} questions')
selected_data = selected_data[~selected_data['question_id'].isin(question_filter)]

filter 4208 questions


In [36]:
# filter students
n_questions = selected_data.groupby('student_id')['question_id'].count()
student_filter = n_questions[n_questions < 20].index.tolist()
print(f'filter {len(student_filter)} students')
selected_data = selected_data[~selected_data['student_id'].isin(student_filter)]

filter 3258 students


In [37]:
# get question to knowledge map
q2k = {}
table = selected_data.loc[:, ['question_id', 'knowledge_id']].drop_duplicates()
for i, row in table.iterrows():
    q = row['question_id']
    # ép về chuỗi để đảm bảo xử lý nhất quán
    knowledge_str = str(row['knowledge_id'])
    # nếu chứa dấu phẩy -> tách lấy id đầu tiên, ngược lại chuyển trực tiếp
    if ',' in knowledge_str:
        first_knowledge = int(knowledge_str.split(',')[0])
    else:
        first_knowledge = int(knowledge_str)
    q2k[q] = {first_knowledge}

    
# get knowledge to question map
k2q = {}
for q, ks in q2k.items():
    for k in ks:
        k2q.setdefault(k, set())
        k2q[k].add(q)

In [38]:
# filter knowledges
#selected_knowledges = { k for k, q in k2q.items()}
selected_knowledges = { k for k, q in k2q.items() if len(q) >= 10}
print(f'filter {len(k2q) - len(selected_knowledges)} knowledges')

filter 77 knowledges


In [39]:
# update maps
q2k = {q : ks for q, ks in q2k.items() if ks & selected_knowledges}
k2q = {}
for q, ks in q2k.items():
    for k in ks:
        k2q.setdefault(k, set())
        k2q[k].add(q)

In [40]:
# update data
selected_data = selected_data[selected_data.apply(lambda x: x['question_id'] in q2k, axis=1)]

In [41]:
# renumber the students
s2n = {}
cnt = 0
for i, row in selected_data.iterrows():
    if row.student_id not in s2n:
        s2n[row.student_id] = cnt
        cnt += 1
selected_data.loc[:, 'student_id'] = selected_data.loc[:, 'student_id'].apply(lambda x: s2n[x])

In [42]:
# renumber the questions
q2n = {}
cnt = 0
for i, row in selected_data.iterrows():
    if row.question_id not in q2n:
        q2n[row.question_id] = cnt
        cnt += 1
selected_data.loc[:, 'question_id'] = selected_data.loc[:, 'question_id'].apply(lambda x: q2n[x])

In [43]:
# renumber the knowledges
k2n = {}
cnt = 0
for i, row in selected_data.iterrows():
    for k in str(row.knowledge_id).split('_'):
        k = str(k)
        if "," in k: k = k.split(",")[0]
        k=int(k)
        if int(float(k)) not in k2n:
            k2n[int(float(k))] = cnt
            cnt += 1
selected_data.loc[:, 'knowledge_id'] = selected_data.loc[:, 'knowledge_id'].apply(lambda x: '_'.join(map(lambda y: str(k2n[int(float(y))]), str(x).split('_'))))


C:\Users\Tung\AppData\Local\Temp\ipykernel_30200\2096838122.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... '26' '23' '25']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  selected_data.loc[:, 'knowledge_id'] = selected_data.loc[:, 'knowledge_id'].apply(lambda x: '_'.join(map(lambda y: str(k2n[int(float(y))]), str(x).split('_'))))


In [44]:
stat_unique(selected_data, None)
a=stat_unique(selected_data, ['student_id', 'question_id'])
b=stat_unique(selected_data, 'student_id')
c=stat_unique(selected_data, 'question_id')
d=stat_unique(selected_data, 'knowledge_id')
print('Average #questions per knowledge: {}'.format((len(q2k) / len(k2q))))

Total length: 447083
Number of unique [student_id,question_id]: 415510
Number of unique student_id: 4719
Number of unique question_id: 2105
Number of unique knowledge_id: 78
Average #questions per knowledge: 26.987179487179485


In [45]:
selected_data.to_csv('selected_data.csv', index=False)

In [46]:
# save concept map
q2k = {}
table = selected_data.loc[:, ['question_id', 'knowledge_id']].drop_duplicates()
for i, row in table.iterrows():
    q = str(row['question_id'])
    q2k[q] = list(map(int, str(row['knowledge_id']).split('_')))
with open('concept_map.json', 'w') as f:
    json.dump(q2k, f)

## parse data

In [47]:
def parse_data(data):
    """ 

    Args:
        data: list of triplets (sid, qid, score)
        
    Returns:
        student based datasets: defaultdict {sid: {qid: score}}
        question based datasets: defaultdict {qid: {sid: score}}
    """
    stu_data = defaultdict(lambda: defaultdict(dict))
    ques_data = defaultdict(lambda: defaultdict(dict))
    for i, row in data.iterrows():
        sid = row.student_id
        qid = row.question_id
        correct = row.correct
        stu_data[sid][qid] = correct
        ques_data[qid][sid] = correct
    return stu_data, ques_data

In [48]:
data = []
for i, row in selected_data.iterrows():
    data.append([row.student_id, row.question_id, row.correct])

In [49]:
stu_data, ques_data = parse_data(selected_data)

In [50]:
test_size = 0.2
least_test_length=150

In [51]:
n_students = len(stu_data)
if isinstance(test_size, float):
    test_size = int(n_students * test_size)
train_size = n_students - test_size
assert(train_size > 0 and test_size > 0)

students = list(range(n_students))
random.shuffle(students)
if least_test_length is not None:
    student_lens = defaultdict(int)
    for t in data:
        student_lens[t[0]] += 1
    students = [student for student in students
                if student_lens[student] >= least_test_length]
test_students = set(students[:test_size])

train_data = [record for record in data if record[0] not in test_students]
test_data = [record for record in data if record[0] in test_students]

In [52]:
def renumber_student_id(data):
    """

    Args:
        data: list of triplets (sid, qid, score)
    
    Returns:
        renumbered datasets: list of triplets (sid, qid, score)
    """
    student_ids = sorted(set(t[0] for t in data))
    renumber_map = {sid: i for i, sid in enumerate(student_ids)}
    data = [(renumber_map[t[0]], t[1], t[2]) for t in data]
    return data

In [53]:
train_data = renumber_student_id(train_data)
test_data = renumber_student_id(test_data)
all_data = renumber_student_id(data)

In [54]:
print(f'train records length: {len(train_data)}')
print(f'test records length: {len(test_data)}')
print(f'all records length: {len(all_data)}')

train records length: 201829
test records length: 245254
all records length: 447083


## save data

In [55]:
def save_to_csv(data, path):
    """

    Args:
        data: list of triplets (sid, qid, correct)
        path: str representing saving path
    """
    pd.DataFrame.from_records(sorted(data), columns=['student_id', 'question_id', 'correct']).to_csv(path, index=False)

In [56]:
save_to_csv(train_data, 'train_triples.csv')
save_to_csv(test_data, 'test_triples.csv')
save_to_csv(all_data, 'triples.csv')

In [57]:
metadata = {"num_students": n_students, 
            "num_questions": c,
            "num_concepts": d, 
            "num_records": len(all_data), 
            "num_train_students": n_students - len(test_students), 
            "num_test_students": len(test_students)}

In [58]:
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)